## Llamaindex function calling agents on Timisoara Public Transport Data

In [1]:
import http.client
import json
import pandas as pd
import os

from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.core.tools import FunctionTool


In [2]:
ratt_key = "fill in your key"

OPENAI_API_KEY  = os.environ['OPENAI_API_KEY']

### Get data about stops

In [3]:
conn = http.client.HTTPSConnection("api.tranzy.ai")

headers = {
    'X-Agency-Id': "8",
    'Accept': "application/json",
    'X-API-KEY': ratt_key
}

conn.request("GET", "/v1/opendata/stops", headers=headers)

res = conn.getresponse()
data = res.read()



In [4]:
df_stop = pd.DataFrame(json.loads(data))

In [5]:
df_stop.head()

,stop_id,stop_name,stop_lat,stop_lon,location_type,stop_code
0,1,Dâmbovița,45.74015,21.19611,0,1
1,2,Dâmbovița,45.74012,21.19624,0,2
2,3,Service Pădureni,45.59412,21.21272,0,3
3,4,Banatul,45.73662,21.19784,0,4
4,5,Banatul,45.73674,21.19787,0,5


## Get data about routes

In [6]:
conn = http.client.HTTPSConnection("api.tranzy.ai")

headers = {
    'X-Agency-Id': "8",
    'Accept': "application/json",
    'X-API-KEY': ratt_key
}

conn.request("GET", "/v1/opendata/routes", headers=headers)

res_routes = conn.getresponse()
data_routes = res_routes.read()
df_routes = pd.DataFrame(json.loads(data_routes))
#json.loads(data_routes)

In [7]:
df_routes.head()

,agency_id,route_id,route_short_name,route_long_name,route_color,route_type,route_desc
0,8,1,1,Gara de Nord - Stația Meteo,#e3a900,0,V. Economu
1,8,2,11,Gheorghe Barițiu - Arena Aqua,#6f2095,11,P-ța 700
2,8,3,33,Catedrala Mitropolitană - Pod Calea Șagului,#0148a2,3,P-ța Mocioni
3,8,6,23,P-ța Gheorghe Domășneanu (Auchan) - Gara de Est,#0148a2,3,P-ța Gheorghe Domășneanu (Auchan) - Gara de Est
4,8,15,2,Shopping City - Stația Meteo,#e3a900,0,P-ța Traian


In [8]:
vehicle_type = {0:"Tram",
1: "Metro",
2 : "Rail",
3 : "Bus",
4 : "Ferry",
5 : "Cable tram",
6 : "Aerial lift",
7 : "Funicular",
11 : "Trolleybus",
12 : "Monorail"}

In [9]:
my_lat, my_long = 45.765697699747, 21.21196413909974 # some location as input

### Write some functions. For the llamaindex framework, the description should be detailed

In [10]:
def get_closest_vehicle(long=my_long, lat=my_lat):
    '''
    Get the closest vehicle to a given location
    The input is the logitude and latitude of the location
    The output is information about the closest vehicle
    '''
    conn.request("GET", "/v1/opendata/vehicles", headers=headers)
    res3 = conn.getresponse()
    data3 = res3.read()
    vehicle_df = pd.DataFrame(json.loads(data3))
    long_sq = [ (x - long)**2 for x in vehicle_df['longitude'].tolist()]
    lat_sq = [(y - lat)**2 for y in vehicle_df['latitude'].tolist()]
    result = [x + y for x, y in zip(long_sq, lat_sq)]
    ind = result.index(min(result))
    row = vehicle_df.iloc[ind,:]
    return row
    #return vehicle_df.iloc[ind,:]["vehicle_type"]



In [11]:
#above, trip_id _0 is the direct route, _1 is the return


In [12]:
def get_closest_vehicle_info(long=my_long, lat=my_lat):
    '''
    Get information about the closest vehicle to a given location
    The input is the logitude and latitude of the location
    The output is vehicle type, route name, starting point name, destination name of the closest vehicle
    '''
    conn.request("GET", "/v1/opendata/vehicles", headers=headers)
    res3 = conn.getresponse()
    data3 = res3.read()
    vehicle_df = pd.DataFrame(json.loads(data3))
    long_sq = [ (x - long)**2 for x in vehicle_df['longitude'].tolist()]
    lat_sq = [(y - lat)**2 for y in vehicle_df['latitude'].tolist()]
    result = [x + y for x, y in zip(long_sq, lat_sq)]
    ind = result.index(min(result))
    row = vehicle_df.iloc[ind,:]
    route_row = df_routes[df_routes["route_id"] == row["route_id"]]
    try:
        way = int(row['trip_id'].split('_')[1])
    except:
        way = 0
    line_ends = elem = route_row["route_long_name"].loc[route_row["route_long_name"].index[0]].split(' - ')
    
    return vehicle_type[row["vehicle_type"]], line_ends[way % 2], line_ends[(1 + way) % 2]



In [13]:
def get_closest_vehicle_location(long=my_long, lat=my_lat):
    '''
    Get the current location of closest vehicle to a given location
    The input is the logitude and latitude of the location
    The output is vehicle's longitude, vehicle's latitude
    '''
    conn.request("GET", "/v1/opendata/vehicles", headers=headers)
    res3 = conn.getresponse()
    data3 = res3.read()
    vehicle_df = pd.DataFrame(json.loads(data3))
    long_sq = [ (x - long)**2 for x in vehicle_df['longitude'].tolist()]
    lat_sq = [(y - lat)**2 for y in vehicle_df['latitude'].tolist()]
    result = [x + y for x, y in zip(long_sq, lat_sq)]
    ind = result.index(min(result))
    row = vehicle_df.iloc[ind,:]
    return row["longitude"], row["latitude"]

In [14]:
def get_closest_stop(long=my_long, lat=my_lat):
    '''
    Get the closest stop to a given location
    The input is the logitude and latitude of the location
    The output is information about the stop
    '''
    long_sq = [ (x - long)**2 for x in df_stop['stop_lon'].tolist()]
    lat_sq = [(y - lat)**2 for y in df_stop['stop_lat'].tolist()]
    result = [x + y for x, y in zip(long_sq, lat_sq)]
    ind = result.index(min(result))
    return df_stop.iloc[ind,:]

In [15]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

### Register the functions as tools

In [16]:
get_closest_stop_tool = FunctionTool.from_defaults(fn=get_closest_stop)
get_closest_vehicle_info_tool = FunctionTool.from_defaults(fn= get_closest_vehicle_info)
get_closest_vehicle_location_tool = FunctionTool.from_defaults(fn= get_closest_vehicle_location)

In [17]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [get_closest_stop_tool, get_closest_vehicle_info_tool, get_closest_vehicle_location_tool ], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

### The agents are able to run the function, even to compose $f\circ g$

In [18]:
response = agent.chat(
    "Which in the closest stop to my location?"
)

Added user message to memory: Which in the closest stop to my location?
=== Calling Function ===
Calling function: get_closest_stop with args: {}
=== Function Output ===
stop_id                875
stop_name          Amforei
stop_lat         45.765295
stop_lon         21.212852
location_type            0
stop_code              898
Name: 874, dtype: object
=== LLM Response ===
The closest stop to your location is "Amforei" with a stop ID of 875. The latitude is 45.765295 and the longitude is 21.212852.


In [19]:
response = agent.chat(
    "What vehicle is closest to my location?"
)

Added user message to memory: What vehicle is closest to my location?
=== Calling Function ===
Calling function: get_closest_vehicle_info with args: {}
=== Function Output ===
Encountered error: Remote end closed connection without response
=== LLM Response ===
I encountered an error while trying to retrieve information about the closest vehicle to your location. Let me try again.
=== Calling Function ===
Calling function: get_closest_vehicle_info with args: {}
=== Function Output ===
('Tram', 'Ronaț', 'Stația Meteo')
=== LLM Response ===
The closest vehicle to your location is a Tram with the route name "Ronaț". It is currently at the starting point named "Stația Meteo".


In [20]:
response = agent.chat(
    "my location has 45.80724372409756 latitude and  21.24189921257169 longitude"
    "Which stop is closest to my location?"
)

Added user message to memory: my location has 45.80724372409756 latitude and  21.24189921257169 longitudeWhich stop is closest to my location?
=== Calling Function ===
Calling function: get_closest_stop with args: {"lat": 45.80724372409756, "long": 21.24189921257169}
=== Function Output ===
stop_id                            607
stop_name        Crișurilor Dumbrăvița
stop_lat                     45.806834
stop_lon                     21.237898
location_type                        0
stop_code                          617
Name: 606, dtype: object
=== LLM Response ===
The closest stop to your location with latitude 45.80724372409756 and longitude 21.24189921257169 is "Crișurilor Dumbrăvița" with a stop ID of 607. The latitude of the stop is 45.806834 and the longitude is 21.237898.


In [21]:
response = agent.chat(
    "my location has 45.80724372409756 latitude and  21.24189921257169 longitude"
    "Tell me which is the closest stop, then what vehicle is closest to that stop"
)

Added user message to memory: my location has 45.80724372409756 latitude and  21.24189921257169 longitudeTell me which is the closest stop, then what vehicle is closest to that stop
=== Calling Function ===
Calling function: get_closest_stop with args: {"lat": 45.80724372409756, "long": 21.24189921257169}
=== Function Output ===
stop_id                            607
stop_name        Crișurilor Dumbrăvița
stop_lat                     45.806834
stop_lon                     21.237898
location_type                        0
stop_code                          617
Name: 606, dtype: object
=== Calling Function ===
Calling function: get_closest_vehicle_info with args: {"lat": 45.80724372409756, "long": 21.24189921257169}
=== Function Output ===
Encountered error: Remote end closed connection without response
=== LLM Response ===
The closest stop to your location with latitude 45.80724372409756 and longitude 21.24189921257169 is "Crișurilor Dumbrăvița" with a stop ID of 607. The latitude of the

In [22]:
response = agent.chat(
    "my location has 45.80724372409756 latitude and  21.24189921257169 longitude"
    "What vehicle is closest to my position and using that vehicle's longitude and latitude"
    " find what stop is closest to the vehicle's position?"
)

Added user message to memory: my location has 45.80724372409756 latitude and  21.24189921257169 longitudeWhat vehicle is closest to my position and using that vehicle's longitude and latitude find what stop is closest to the vehicle's position?
=== Calling Function ===
Calling function: get_closest_vehicle_info with args: {"lat": 45.80724372409756, "long": 21.24189921257169}
=== Function Output ===
('Bus', 'Ion Ionescu de la Brad', 'Belvedere (Dumbrăvița)')
=== Calling Function ===
Calling function: get_closest_vehicle_location with args: {"lat": 45.80724372409756, "long": 21.24189921257169}
=== Function Output ===
(21.2348633, 45.78039)
=== Calling Function ===
Calling function: get_closest_stop with args: {"lat": 45.78039, "long": 21.2348633}
=== Function Output ===
stop_id                             162
stop_name        Ion Ionescu de la Brad
stop_lat                       45.77995
stop_lon                       21.23458
location_type                         0
stop_code            